# X-ray Absorption Spectroscopy (XAS) Experiment at SCS

Author: Jun Zhu jun.zhu@xfel.eu

Created on Nov. 12, 2018

Copyright (C) European X-Ray Free-Electron Laser Facility GmbH.

All rights reserved.

## Introduction

<table><tr>
    <td>
        <img src='misc/mono.png' width="600 px">
        <figcaption>Higley et al., Rev. Sci. Instrum.  87, 033110 (2016)</figcaption>
    </td>
    <td>
        <img src='misc/illustration.png' width="920 px">
    </td>
    <td>
        <img src='misc/expected_data.png' width="800 px">
        <figcaption>Higley et al., submitted</figcaption></td>
</tr></table>

## Data Analysis

In [ ]:
import os
import numpy as np
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
try:
    # for interactive plot
    from plotly.offline import init_notebook_mode, iplot
    import plotly.graph_objs as go
    init_notebook_mode(connected=True)
    
    USE_PLOTLY = True
except ImportError:
    USE_PLOTLY = False

from xas_processor import XasDigitizer

In [ ]:
!ls /gpfs/exfel/exp/SCS/201831/p900048/raw

In [ ]:
run_folder = "/gpfs/exfel/exp/SCS/201831/p900048/raw/r0313"

xas = XasDigitizer(run_folder, pulse_id_min=30, n_pulses=10)
xas.info()

In [ ]:
config = {"sample_start": 14590, "peak_width": 30, "background_end": 14580, "background_width": 500}
xas.process(config)

xas.correlation('mcp2').head()

In [ ]:
%load_ext autoreload
%autoreload 2
fig, ax = xas.plot_correlation('mcp1', marker_size=6, alpha=0.05, n_bins=20)

In [ ]:
raise ValueError

## Sanity check

***check train resolved parameters throughtout the run***

```py
def plot_xgm(self, *, figsize=(8, 7.2)):
    """Plot the train resolved data from XGM.
    
    :param tuple figsize: figure size.
    """
```

In [ ]:
fig, (ax11, ax12, ax2) = xas.plot_xgm_run()

***check pulse resolved XGM signals***

why negative pulse energies?

In [ ]:
fig, ax = xas.plot_xgm_train(index=110)

***check digitizer signals***

In [ ]:
fig, axes = xas.plot_digitizer_train(index=0, x_min=14000, x_max=38000)

## Correlation between I0 and I1

In [ ]:
xas.process()
corrs = []
for name in adc_channel_names:
    # 'XasFastADC.correlation()' returns a panda.DataFrame object
    corrs.append(xas.correlation(name))

corrs[0].head()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for ax, corr, name in zip(np.ravel(axes), corrs, adc_channel_names):
    corr.plot.scatter(x="XGM", y="MCP", ax=ax, alpha=0.05)

    ax.set_xlabel(r"$I_0$ ($\mu$ J)")
    ax.set_ylabel(r"$I_1$ (arb.)")
    ax.set_title(name)

fig.suptitle(run_folder)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

## Absorption spectrum

In [ ]:
xas = XasFastADC(xgm_id=xgm_device, adc_id=adc_device, adc_channels=adc_channels)

# Use case without softmono in SCS DAQ data
for run_number, energy in run_energies.items():
    xas.process_run("/gpfs/exfel/exp/SCS/201831/p900048/raw/r{:04d}".format(run_number), photon_energy=energy)

In [ ]:
# 'XasFastADC.spectrum()' returns a panda.DataFrame object
spectrums = dict()
for name in adc_channel_names:
    spectrums[name] = xas.spectrum(name)

spectrums["MCP1"].head()

In [ ]:
ax = spectrums["MCP1"].plot.scatter(x="photon_energy", y="weight")
ax.set_xlabel("Photon energy (ev)")
ax.set_ylabel(r"Weight ($\mu$J)")

In [ ]:
# plot the "masked" spectrum

selected_channels = ["MCP1", "MCP2", "MCP3"]
colors = ['r', 'g', 'b']

fig, ax = plt.subplots(figsize=(8, 6))
for name, color in zip(selected_channels, colors):
    sp = spectrums[name]
    # masked by applying a lower boundary to weight
    sp[sp['weight'] > 300].plot.scatter(x="photon_energy", y="absorption", s=40, c=color, ax=ax)

ax.set_xlabel("Photon energy (ev)")
ax.set_ylabel("absorption (arb.)")  # -log(I1/I0)    
ax.legend(selected_channels)